In [1]:
# KNN algorithm
import pandas as pd
import numpy as np
import math 
import random
import csv

In [2]:
heart = pd.read_csv("D:\Fall 2020 courses\CSC5825 machine learning\homework\HW2\heart.csv")

In [3]:
heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Normalizing data using function F(X) = (X 􀀀 mean)=std
heart[['age','trestbps','chol','thalach','oldpeak']] = heart[['age','trestbps','chol','thalach','oldpeak']].apply(lambda data: abs(data - np.mean(data)) / np.std(data), axis = 0)

In [5]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature.
columnTransformer = ColumnTransformer([('encoder', 
                                         OneHotEncoder(), 
                                         [2,6,10,11,12])], 
                                       remainder='passthrough') 
  
data = np.array(columnTransformer.fit_transform(heart), dtype = np.str) 

In [6]:
heart_df=pd.DataFrame(data)   
heart_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.9521966015692286,1.0,0.7639557707131769,0.25633370748033474,1.0,0.015442785566123276,0.0,1.0873380612201569,1.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.9153128889362694,1.0,0.09273778415763291,0.07219949163168411,0.0,1.633471467639158,0.0,2.1225727275193167,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.4741575827046542,0.0,0.09273778415763291,0.8167726942008375,0.0,0.9775138938257655,0.0,0.31091206149578715,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.18017481566390225,1.0,0.6638668207381728,0.19835726057821376,0.0,1.2398969233511226,0.0,0.20670527165379263,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.29046364222180604,0.0,0.6638668207381728,2.082049650905211,0.0,0.5839393495377301,1.0,0.3792443827036526,1.0


In [7]:
# Splitting the data and seperating the target values - column 27

train, validate, test = np.split(heart_df.sample(frac=1), [int(.6*len(heart_df)), int(.8*len(heart_df))])
train = train.values.tolist()
validate_output = validate[27].tolist()
validate = validate.values.tolist()
test_output = test[27].tolist()
test = test.values.tolist()

In [8]:
#calculating euclidian distance
def euc_dist(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += pow((float(row1[i]) - float(row2[i])),2)
    return np.sqrt(distance)

In [9]:
# locating similar neighbours
def get_neighbours(train, test_row, num_neighbours):
    distances = list()
    for train_row in train:
        dist = euc_dist(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key = lambda tup:tup[1])
    neighbours = list()
    for i in range(num_neighbours):
        neighbours.append(distances[i][0])
    return neighbours


In [10]:
# predicton with neighbours
def predict_neighbours(train,test_row, num_neighbours):
    neighbours = get_neighbours(train,test_row, num_neighbours)
    output_values = [row[-1] for row in neighbours]
    prediction = max(set(output_values), key = output_values.count)
    return prediction

In [11]:
# calculate accuracy
def accuracy_rate(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [12]:
# # KNN algo
def knn(train, test, num_neighbours):
        predictions = list()
        for row in test:
            output = predict_neighbours(train, row, num_neighbours)
            predictions.append(output)
        return(predictions)

In [13]:
# make prediction in data
num_neighbours = [3,5] 
for n in num_neighbours:
    predictions = knn(train, validate, n)
    accuracy = accuracy_rate(predictions, validate_output)
    print('n = {0}, accuracy = {1}'.format(n, accuracy))

n = 3, accuracy = 81.9672131147541
n = 5, accuracy = 86.88524590163934


In [14]:
predictions = knn(train, test, 3)
accuracy = accuracy_rate(predictions, test_output)
print('n = 3, accuracy = {1}'.format(n, accuracy))

n = 3, accuracy = 73.77049180327869


In [15]:
# CONFUSION METRIx
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
confusion_matrix(predictions, test_output)

array([[18,  6],
       [10, 27]], dtype=int64)

In [16]:
accuracy = accuracy_score(predictions, test_output)
print('n = 3, accuracy = {1}'.format(n, accuracy))

n = 3, accuracy = 0.7377049180327869


In [17]:
precision = precision_score(predictions, test_output, pos_label="1.0")
print('n = 3, precision = {1}'.format(n, precision))

n = 3, precision = 0.8181818181818182


In [18]:
recall = recall_score(predictions, test_output, pos_label="1.0")
print('n = 3, recall = {1}'.format(n, recall))

n = 3, recall = 0.7297297297297297


In [19]:
f1 = f1_score(predictions, test_output, pos_label="1.0")
print('n = 3, f1 = {1}'.format(n, f1))

n = 3, f1 = 0.7714285714285715
